In [21]:
#이미지 -> 배열화하는 방법은 도저히 일반화시킬 수가 없을 듯함
#if _____ in _____ 문법이기 때문에 해당 부분을 모든 데이터에 일반화시킬 수 없을 듯하여 이를 고민해봐야 할 듯 함
# learning_rate 적용은 optimizer에 해주었으며, optimizer 부분에 learning_rate 외 여러 파라미터 존재 -> 어떤 부분이 필요하고 어떤 부분이 불필요한지 판단
# https://keras.io/ko/optimizers/
# 사용자 지정 파라미터 : batch_size, optimizer, batch_size, epochs, loss 함수

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

In [22]:
import os, re, glob
import cv2
import numpy as np
#from keras.preprocessing import image
from PIL import Image

In [23]:
groups_folder_path = './resources/imgs300x300/'

In [24]:
def category_set(): #해당 카테고리의 경우 이미지 이름에서 카테고리를 분류할 수 있는 이름으로 입력받아야 함
    categories = input().split()
    print(categories)
    
    return categories

In [26]:
def image_to_list(groups_folder_path):
    data_path = os.listdir(groups_folder_path)
    
    X = []
    Y = []
    categories = category_set()
    
    for filepath in data_path:
        #print(filepath)
        filepath = groups_folder_path + filepath
        file_path = os.listdir(filepath)
        # print(file_path)

        for data in file_path:
            path_file = filepath + '/' + data
            #print(path_file)
            # print(data)
            # img = img = image.load_img(path_file, target_size=(28, 28,1))
            image = Image.open(path_file)
            image = image.convert('L')  # 흑백으로 맹글기
            image = image.resize((28, 28))
            img_data = np.array(image)
            img_data = img_data.reshape((28, 28, 1))

            #img = cv2.resize(img, None, fx=image_w/img.shape[1], fy=image_h.shape[0])
            # img_data = image.img_to_array(img)
            # img_data = np.expand_dims(img_data, axis=0)
            # img_data = preprocess_input(img_data)
            # X.append(np.array(img_data))
            img_data_np = np.array(img_data)
            
            for i in range(len(categories)):
                if categories[i] in data:
                    X.append(img_data_np)
            
            for i in range(len(categories)):
                if categories[i] in data:
                    Y.append(categories[i])
                    
    X = np.array(X)
    Y = np.array(Y)
                    
    return X, Y

In [27]:
def split_data(groups_folder_path, X, Y):
    X, Y = image_to_list(groups_folder_path)
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
    xy = (X_train, X_test, Y_train, Y_test)
    np.save("./img_data.npy", xy)
    X_train, X_test, Y_train, Y_test = np.load('./img_data.npy', allow_pickle = True)
    
    return X_train, X_test, Y_train, Y_test

In [14]:
from keras.models import Sequential
from keras.layers import Dropout, Activation, Dense
from keras.layers import Flatten, Convolution2D, MaxPooling2D
from keras.models import load_model
from keras import optimizers
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D
from keras import backend as K

In [9]:
def normalization_tool(data):
    data_nor = data.astype('float32') / 255
    
    return data_nor

In [16]:
batch_size = 32
learning_rate = 0.01
epochs = 500

In [17]:
optimizer='Adam'
loss='binary_crossentropy'

In [18]:
def optimizer_set(optimizer, learning_rate):
    if optimizer == 'SGD':
        optimizer = optimizers.SGD(lr=learning_rate, clipnorm=1.)
        
    if optimizer == 'RMSprop':
        optimizer = optimizers.RMSprop(lr=learning_rate, rho=0.9, epsilon=None, decay=0.0)
        
    if optimizer == 'Adam':
        optimizer = optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        
    if optimizer == 'Nadam':
        optimizer = optimizers.Nadam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
        
    if optimizer == 'Adagrad':
        optimizer = optimizers.Adagrad(lr=learning_rate, epsilon=None, decay=0.0)
        
    if optimizer == 'Adadelta':
        optimizer = optimizers.Adadelta(lr=learning_rate, rho=0.95, epsilon=None, decay=0.0)
        
    return optimizer

In [28]:
def twentyeightCNNAutoEncoder(groups_folder_path, X, Y, batch_size, learning_rate, epochs, optimizer, loss):
    X_train, X_test, Y_train, Y_test = split_data(groups_folder_path, X, Y) #데이터 분리
    X_train = normalization_tool(X_train) #데이터 정규화
    X_test = normalization_tool(X_test) #데이터 정규화
    
    optimizer = optimizer_set(optimizer, learning_rate) #optimizer 설정
    
    input_shape = (28, 28, 1)
    model = Sequential()
    
    # 1st convolution layer
    model.add(Conv2D(16, (3, 3)  # 16 is number of filters and (3, 3) is the size of the filter.
                 , padding='same', input_shape=(28, 28, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

    # 2nd convolution layer
    model.add(Conv2D(2, (3, 3), padding='same'))  # apply 2 filters sized of (3x3)
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

    # here compressed version

    # 3rd convolution layer
    model.add(Conv2D(2, (3, 3), padding='same'))  # apply 2 filters sized of (3x3)
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))

    # 4rd convolution layer
    model.add(Conv2D(16, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))

    model.add(Conv2D(1, (3, 3), padding='same'))
    model.add(Activation('sigmoid'))
    model.summary()
    
    #학습(약 7000번 정도 진행)
    model.compile(optimizer=optimizer, loss=loss) #사용자 지정 파라미터(optimizer, loss)
    model.fit(X_train, X_train, batch_size = batch_size, epochs=epochs, validation_data=(X_test, X_test)) #사용자 지정 파라미터(epochs)
    
    X = normalization_tool(X) #데이터 정규화
    
    compressed_layer = 5
    get_3rd_layer_output = K.function([model.layers[0].input],[model.layers[compressed_layer].output])
    compressed = get_3rd_layer_output([X])[0]
    
    #일렬로 늘리기
    compressed = compressed.reshape(compressed.shape[0], compressed.shape[1]*compressed.shape[2]*compressed.shape[3])
    
    return compressed

In [29]:
def ninetysixCNNAutoEncoder(groups_folder_path, X, Y, batch_size, learning_rate, epochs, optimizer, loss):
    X_train, X_test, Y_train, Y_test = split_data(groups_folder_path, X, Y) #데이터 분리
    X_train = normalization_tool(X_train) #데이터 정규화
    X_test = normalization_tool(X_test) #데이터 정규화
    
    optimizer = optimizer_set(optimizer, learning_rate) #optimizer 설정
    
    input_shape = (96, 96, 1)
    model = Sequential()
    
    # 1st convolution layer
    model.add(Conv2D(16, (3, 3)  # 16 is number of filters and (3, 3) is the size of the filter.
                 , padding='same', input_shape=(96, 96, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

    # 2nd convolution layer
    model.add(Conv2D(2, (3, 3), padding='same'))  # apply 2 filters sized of (3x3)
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

    # here compressed version

    # 3rd convolution layer
    model.add(Conv2D(2, (3, 3), padding='same'))  # apply 2 filters sized of (3x3)
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2), padding='same'))

    # 4rd convolution layer
    model.add(Conv2D(16, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))
    
    #5th convolution layer
    model.add(Conv2D(2,(3, 3), padding='same')) # apply 2 filters sized of (3x3)
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))
 
    #6th convolution layer
    model.add(Conv2D(16,(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))

    model.add(Conv2D(1, (3, 3), padding='same'))
    model.add(Activation('sigmoid'))
    model.summary()
    
    #학습(약 7000번 정도 진행)
    model.compile(optimizer=optimizer, loss=loss) #사용자 지정 파라미터(optimizer, loss)
    model.fit(X_train, X_train, batch_size = batch_size, epochs=epochs, validation_data=(X_test, X_test)) #사용자 지정 파라미터(epochs)
    
    X = normalization_tool(X) #데이터 정규화
    
    compressed_layer = 8
    get_3rd_layer_output = K.function([model.layers[0].input],[model.layers[compressed_layer].output])
    compressed = get_3rd_layer_output([X])[0]
    
    #일렬로 늘리기
    compressed = compressed.reshape(compressed.shape[0], compressed.shape[1]*compressed.shape[2]*compressed.shape[3])
    
    return compressed